In [1]:
# Install required libraries
# You can install these packages using pip if not installed:
# !pip install pandas sqlite3 pandasql

import sqlite3
import pandas as pd

# Set working directory
import os
os.chdir("/Users/jade/Desktop/Humana/Training")

# Load datasets using pandas
features_data = pd.read_csv("Afeatures.csv")
control_data = pd.read_csv("Controlpoint.csv")
cost_data = pd.read_csv("CostUt.csv")
demograph_data = pd.read_csv("Demographics.csv")
condition_data = pd.read_csv("mcondition.csv")
detail_data = pd.read_csv("mdetail.csv")
claims_data = pd.read_csv("mclaims.csv")
target_data = pd.read_csv("Tmembers.csv")
members_data = pd.read_csv("mdata.csv")
pharmacy_data = pd.read_csv("PU.csv")
quality_data = pd.read_csv("QD.csv")
sales_data = pd.read_csv("SC.csv")
social_data = pd.read_csv("Socialh.csv")
web_data = pd.read_csv("WA.csv")

/var/folders/wm/7c949vv90fl9xprhlvgygqw40000gn/T/ipykernel_52273/3976071838.py:19: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_data = pd.read_csv("mclaims.csv")
/var/folders/wm/7c949vv90fl9xprhlvgygqw40000gn/T/ipykernel_52273/3976071838.py:23: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  quality_data = pd.read_csv("QD.csv")


In [2]:
import re
def convert_tenure(value):
    # Check if value is a string before processing
    if isinstance(value, str):
        # If the format is 'X - Y YEARS', calculate the midpoint
        if '-' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 2:  # Ensure there are two numbers for the range
                return (float(numbers[0]) + float(numbers[1])) / 2
        # If the format is 'X+ YEARS', take the number before the '+'
        elif '+' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:  # Ensure there's at least one number
                return float(numbers[0])
        # If there's just 'X YEARS', return the number
        elif 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:
                return float(numbers[0])
        # Default case for strings that don't match the expected format
        return None
    # If value is already a float or an int, return it as is
    elif isinstance(value, (float, int)):
        return value
    # Default return for unexpected types (e.g., NaN, None)
    return None

# Applying the function to the column
members_data['tenure_band'] = members_data['tenure_band'].apply(convert_tenure)
members_data['disabled_ind'] = members_data['disabled_ind'].replace({'Y': 1, 'N': 0})
members_data['dual_eligible_ind'] = members_data['dual_eligible_ind'].replace({'Y': 1, 'N': 0})
members_data['lis_ind']= members_data['lis_ind'].replace({'Y': 1, 'N': 0})

In [3]:
merged_data = pd.merge(members_data, pharmacy_data, on='id', how='inner')
merged_data = pd.merge(merged_data, social_data, on='id', how='inner')
merged_data = pd.merge(merged_data, web_data, on='id', how='inner')

In [4]:
new_target=target_data.drop(['product_type','calendar_year','plan_category'],axis=1)
merged_data = pd.merge(merged_data, new_target, on='id', how='inner')
merged_data = pd.merge(merged_data, features_data, on='id', how='inner')
merged_data = pd.merge(merged_data, control_data, on='id', how='inner')
merged_data = pd.merge(merged_data, cost_data, on='id', how='inner')
demograph_data=demograph_data.drop(columns=['lang_spoken_cd', 'rucc_category'])
merged_data = pd.merge(merged_data, demograph_data, on='id', how='inner')

In [5]:
sales_data['channel'] = sales_data['channel'].map({
    'Field':0,
    'Consumer Direct': 1,
    'Partner Call Center': 2,
    'DMS Telesales': 3,
    'Brokerage': 4
})
merged_data = pd.merge(merged_data, sales_data, on='id', how='inner')
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 231 entries, consec_tenure_month to channel
dtypes: float64(224), int64(7)
memory usage: 2.6 GB


In [6]:
region_mapping = {region: idx for idx, region in enumerate(detail_data['region'].fillna('Unknown').unique())}
race_mapping = {race: idx for idx, race in enumerate(detail_data['race'].fillna('Unknown').unique())}

# Print the mappings to see what was assigned (optional)
print("Region Mapping:", region_mapping)
print("Race Mapping:", race_mapping)

# Map the 'region' column to numeric values and overwrite the original column
detail_data['region'] = detail_data['region'].fillna('Unknown').map(region_mapping)

# Map the 'race' column to numeric values and overwrite the original column
detail_data['race'] = detail_data['race'].fillna('Unknown').map(race_mapping)
detail_data=detail_data.drop(columns=['mco_contract_nbr','state_of_residence','county_of_residence'])
detail_data['generic_grouper'] = detail_data['generic_grouper'].fillna(10008)
detail_data['generic_grouper'] = detail_data['generic_grouper'].replace({'Y': 1, 'N': 0})
detail_data['unattributed_provider'] = detail_data['unattributed_provider'].fillna(10008)
detail_data['unattributed_provider'] = detail_data['unattributed_provider'].replace({'Y': 1, 'N': 0})
detail_data['sex_cd'] = detail_data['sex_cd'].fillna(10008)
detail_data['sex_cd'] = detail_data['sex_cd'].replace({'F': 1, 'M': 0,'U':10008})
detail_data['veteran_ind'] = detail_data['veteran_ind'].fillna(10008)
detail_data['veteran_ind'] = detail_data['veteran_ind'].replace({'Y': 1, 'N': 0})
merged_data = pd.merge(merged_data, detail_data, on='id', how='inner')

Region Mapping: {'Florida': 0, 'Gulf South': 1, 'South Central': 2, 'Southeast': 3, 'Midwest': 4, 'Northeast': 5, 'Intermountain': 6, 'Central': 7, 'Pacific Southwest': 8, 'Puerto Rico': 9, 'Unknown': 10}
Race Mapping: {'Unknown': 0, 'BLACK': 1, 'N AMERICAN NATIVE': 2, 'WHITE': 3, 'ASIAN': 4, 'HISPANIC': 5, 'UNKNOWN': 6, 'OTHER': 7}


In [7]:
condition_aggregated = condition_data.groupby('id').agg({
    'hcc_model_type': lambda x: ', '.join(x),  # Join diseases as a single string
}).reset_index()
condition_aggregated
def remove_duplicates(hcc_model_type):
    # Split the string into a list, remove duplicates using set, and join it back into a string
    unique_terms = ', '.join(sorted(set(hcc_model_type.split(', '))))
    return unique_terms

# Apply the function to remove duplicates from 'cond_desc' column
condition_aggregated['hcc_model_type'] = condition_aggregated['hcc_model_type'].apply(remove_duplicates)
merged_data = pd.merge(merged_data, condition_aggregated, on='id', how='outer')

In [8]:
merged_data['hcc_model_type'] = merged_data['hcc_model_type'].fillna(0)

# Step 2: Replace specific strings with numeric values
merged_data['hcc_model_type'] = merged_data['hcc_model_type'].replace({'U': 0, 'MEDICAL': 1, 'ESRD': 2})
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 241 entries, consec_tenure_month to hcc_model_type
dtypes: float64(224), int64(17)
memory usage: 2.7 GB


In [9]:
# Step 1: Replace 'Y' with 1 and everything else (including NaN) with 0
claims_data_numeric = claims_data.replace({'Y': 1}).fillna(0)

# Step 2: Group by 'id' and sum the numeric values to count the number of 'Y's in each column
claims_data_group = claims_data_numeric.groupby('id', as_index=False).sum()

In [10]:
claims_data_group=claims_data_group.drop(columns = ['dos_year','clm_unique_key','serv_date_skey'])

In [11]:
merged_data = pd.merge(merged_data, claims_data_group, on='id', how='outer')

In [12]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 264 entries, consec_tenure_month to er_visit
dtypes: float64(247), int64(17)
memory usage: 3.0 GB


In [13]:
quality_data = quality_data.drop(columns = ['measurement_year','measure_name','measure_desc','base_event_date','eligible_cnt'])
quality_data['measure_type'] = quality_data['measure_type'].fillna(10008)
quality_data['measure_type'] = quality_data['measure_type'].replace({'Patient Safety': 1, 'Patient Experience': 0, 'HEDIS':2})

In [14]:
quality_data_group=quality_data.groupby('id').sum()

In [15]:
merged_data = pd.merge(merged_data, quality_data_group, on='id', how='outer')

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer

X = merged_data.drop(columns=['preventive_visit_gap_ind'])  # Features
y = merged_data['preventive_visit_gap_ind']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the parameter search space
param_space = {
    'learning_rate': Real(0.01, 1.0, prior='log-uniform'),
    'max_depth': Integer(3, 10),
    'n_estimators': Integer(50, 300),
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.5, 1.0)
}

# Setup the Bayesian Optimization with cross-validation
bayes_search = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=param_space,
    n_iter=32,  # Number of parameter settings that are sampled
    cv=3,       # 3-fold cross-validation
    n_jobs=-1,  # Use all available cores
    random_state=42
)

# Fit the Bayesian Optimization model
bayes_search.fit(X_train, y_train)

# Display the best parameters
print("Best parameters found by Bayesian Optimization:")
print(bayes_search.best_params_)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7, 10],
    'n_estimators': [50, 100, 200],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Setup the Grid Search with cross-validation
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,        # 3-fold cross-validation
    n_jobs=-1,   # Use all available cores
    verbose=1    # Show progress
)

# Fit the Grid Search model
grid_search.fit(X_train, y_train)

# Display the best parameters
print("Best parameters found by Grid Search:")

In [ ]:
merged_data.info()